In [22]:
import pandas as pd
import seaborn as sns
from ipyleaflet import *
from branca.colormap import linear
import json
import pandas
import geopandas as gpd
from colour import Color
from pandas_profiling import ProfileReport
import numpy as np
from ipywidgets import *
import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [24]:
# On importe le data frame que l'on a construit (cf note book get_data)

df_regression = pd.read_csv(r'C:\Users\matth\OneDrive - GENES\ENSAE 2A\Python pour le data scientist/df_regression.csv')
#df_regression = pd.read_csv('../df_regression.csv')


#On ne garde que les données à partir de 1995 pour que la régression soit la plus fiable possible
df_regression = df_regression[df_regression['year']>= 1995]


,country,year,croissancePIB,mortalite-enf,population,export,polity2,ratio-zone,ratio
0,Zimbabwe,2018,4.824211,17413.0,14438812.0,210.771948,4.0,0.002011,0.000000
1,Zimbabwe,2017,4.709492,18293.0,14236599.0,180.807792,4.0,0.000030,0.000000
2,Zimbabwe,2016,0.755869,19029.0,14030338.0,173.230130,4.0,0.000650,0.000000
3,Zimbabwe,2015,1.779873,20014.0,13814642.0,168.739718,4.0,0.000024,0.000000
4,Zimbabwe,2014,2.376929,20871.0,13586710.0,200.850799,4.0,0.000206,0.000000
...,...,...,...,...,...,...,...,...,...
4301,Afghanistan,1999,6.836849,89438.0,20170847.0,121.068078,-7.0,0.001316,0.051293
4302,Afghanistan,1998,6.836849,88705.0,19737770.0,115.958547,-7.0,0.011979,0.198675
4303,Afghanistan,1997,6.836849,87552.0,19357126.0,105.139391,-7.0,0.004240,0.109178
4304,Afghanistan,1996,6.836849,85980.0,18853444.0,93.404801,-7.0,0.040444,0.050224


In [14]:
# On recalibre les données pour effectuer une régression, passage du format long au format wide
df_final = pd.DataFrame()
for cntry in df_regression.country.unique():
    df_temp = df_regression[df_regression.country == cntry]
    df_new = pd.DataFrame({cntry:[]})
    columns = np.array(df_temp.columns)
    for y in range(1990,2019):

        if df_temp[df_temp.year == y].empty:

            for col in columns:
                if col != 'year' and col != 'country':
                    filler = df_temp[col].mean()
                    df_new.loc[str(y) + '-' + col] = filler
        else:

            for col in columns:
                if col != 'year' and col != 'country':
                    df_new.loc[str(y) + '-' + col] = df_temp[df_temp.year == y][col].values

    df_final = pd.concat([df_final, df_new.transpose()])
    
df_final

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-polity2,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,2018-ratio-zone,2018-ratio
Zimbabwe,1.268572,20981.083333,1.256234e+07,136.330378,-1.416667,0.000450,0.000000,1.268572,20981.083333,1.256234e+07,...,4.0,0.000030,0.000000,4.824211,17413.0,14438812.0,210.771948,4.0,0.002011,0.000000
Zambia,5.534573,33454.041667,1.268306e+07,494.755586,5.000000,0.006294,0.000008,5.534573,33454.041667,1.268306e+07,...,6.0,0.000293,0.000000,4.034494,26866.0,17351714.0,1012.443792,6.0,0.002291,0.000000
Yemen,1.120798,40852.250000,2.128825e+07,111.755404,-1.166667,0.000316,0.000000,1.120798,40852.250000,2.128825e+07,...,0.0,0.000040,0.000000,0.752447,37517.0,28498683.0,33.288874,0.0,0.000128,0.000000
Vietnam,6.671504,31087.041667,8.523303e+07,500.452668,-7.000000,0.000000,0.000000,6.671504,31087.041667,8.523303e+07,...,-7.0,0.000000,0.000000,7.075789,25987.0,95545959.0,1682.653459,-7.0,0.000000,0.000000
Uzbekistan,5.771034,22790.041667,2.733835e+07,255.060565,-9.000000,0.132058,0.000215,5.771034,22790.041667,2.733835e+07,...,-9.0,0.199614,0.000000,5.354997,11479.0,32956100.0,387.671272,-9.0,0.312684,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Argentina,2.205179,10587.208333,3.954242e+07,180.912015,8.000000,0.003082,0.000000,2.205179,10587.208333,3.954242e+07,...,9.0,0.028273,0.000000,-2.617396,6560.0,44494502.0,234.543108,9.0,0.027312,0.000000
Angola,6.357488,82248.166667,2.124495e+07,381.830277,-2.958333,0.000045,0.006081,6.357488,82248.166667,2.124495e+07,...,-2.0,0.000000,0.000262,-2.003630,64019.0,30809787.0,514.553299,-2.0,0.000000,0.000279
Algeria,3.416667,21136.541667,3.457893e+07,175.686146,0.125000,0.010172,0.014457,3.416667,21136.541667,3.457893e+07,...,2.0,0.019693,0.000777,1.100000,20875.0,42228415.0,189.716555,2.0,0.024777,0.000407
Albania,4.837915,812.416667,2.994021e+06,435.138961,7.375000,0.000000,0.000003,4.837915,812.416667,2.994021e+06,...,9.0,0.000000,0.000000,4.019360,289.0,2866376.0,1097.548787,9.0,0.000000,0.000000


In [15]:
# On va essayer de prédire le ratio de risque de 2018 en utilisant toutes les données antérieures
df_final.rename(columns = {'2018-ratio':'aim'}, inplace = True)
df_final

# On sépare le jeu de données en entrainement / test

train, test = train_test_split(df_final, test_size = 0.2)
X_train = train.iloc[:, :-2]
y_train = train.iloc[:,-1:]
X_test = test.iloc[:, :-2]
y_test = test.iloc[:,-1:]

In [16]:
scalerX = StandardScaler().fit(X_train)
scalerX_test = StandardScaler().fit(X_test)

X_strain = scalerX.transform(X_train)
X_stest = scalerX_test.transform(X_test)

In [17]:
y_train.values

array([[0.00000000e+00],
       [0.00000000e+00],
       [3.25554897e-04],
       [0.00000000e+00],
       [0.00000000e+00],
       [5.61582197e-03],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [2.90674015e-02],
       [5.06935482e-03],
       [0.00000000e+00],
       [1.10456126e-02],
       [0.00000000e+00],
       [1.17083493e-02],
       [7.09244596e-04],
       [0.00000000e+00],
       [1.66381806e-02],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [2.82535142e-03],
       [3.72062739e-04],
       [0.00000000e+00],
       [0.00000000e+00],
       [2.08122595e-03],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [1.95332938e-03],
       [0.00000000e+00],
       [1.16269606e-05],
       [0.00000000e+00],
       [5.12748962e-03],
       [6.89478763e-03],
       [3.12637343e-01],
       [4.65078424e-05],
       [0.00000000e+00],


In [18]:
# Modèle linéaire 

clf = LinearRegression()

clf.fit(X_strain, y_train.values)

LinearRegression()

In [19]:
# On regarde les prédictions
X_test['aim'] = y_test
X_test['prediction'] = clf.predict(X_stest)

C:\Users\matth\AppData\Local\Temp/ipykernel_3280/3479116683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['aim'] = y_test


In [20]:
X_test

,1990-croissancePIB,1990-mortalite-enf,1990-population,1990-export,1990-polity2,1990-ratio-zone,1990-ratio,1991-croissancePIB,1991-mortalite-enf,1991-population,...,2017-polity2,2017-ratio-zone,2017-ratio,2018-croissancePIB,2018-mortalite-enf,2018-population,2018-export,2018-polity2,aim,prediction
Burundi,1.429327,23063.541667,8.022712e+06,169.566464,2.208333,0.010138,0.013130,1.429327,23063.541667,8.022712e+06,...,-1.0,0.000101,0.000434,1.609933,17719.0,1.117538e+07,360.332539,-1.0,0.001314,0.397389
Malaysia,5.120665,3967.916667,2.630334e+07,157.253108,4.291667,0.011092,0.000027,5.120665,3967.916667,2.630334e+07,...,5.0,0.000525,0.000010,4.843057,3798.0,3.152803e+07,251.916502,7.0,0.000000,-0.032922
Latvia,4.088774,181.041667,2.196774e+06,399.879553,8.000000,0.000000,0.000000,4.088774,181.041667,2.196774e+06,...,8.0,0.000000,0.000000,3.989480,68.0,1.927174e+06,865.191186,8.0,0.000000,-0.021492
Denmark,1.690689,261.333333,5.481254e+06,161.710335,10.000000,0.000005,0.000000,1.690689,261.333333,5.481254e+06,...,10.0,0.000000,0.000000,1.989537,204.0,5.793636e+06,214.422536,10.0,0.000000,-0.004522
Cambodia,7.707682,18408.333333,1.355983e+07,328.234671,1.041667,0.004626,0.000000,7.707682,18408.333333,1.355983e+07,...,-4.0,0.000505,0.000000,7.469169,8642.0,1.624980e+07,914.047766,-4.0,0.000000,-0.018444
Chile,4.210145,2047.708333,1.647372e+07,245.244502,9.333333,0.000722,0.000000,4.210145,2047.708333,1.647372e+07,...,10.0,0.000071,0.000000,3.713859,1433.0,1.872917e+07,391.458711,10.0,0.000000,-0.012891
Suriname,2.881175,256.208333,5.089002e+05,322.821302,5.000000,0.003090,0.000000,2.881175,256.208333,5.089002e+05,...,5.0,0.028273,0.000000,4.948271,177.0,5.759870e+05,537.325863,5.0,0.000000,0.007315
Burkina Faso,6.041803,45145.416667,1.436930e+07,555.447984,-0.083333,0.004217,0.000127,6.041803,45145.416667,1.436930e+07,...,6.0,0.008630,0.000545,6.604569,40798.0,1.975147e+07,1556.772474,6.0,0.002267,-0.033574
Guinea,4.413519,32235.875000,9.576372e+06,212.182622,0.500000,0.023946,0.000935,4.413519,32235.875000,9.576372e+06,...,4.0,0.006107,0.000000,6.358492,28932.0,1.241429e+07,597.307808,4.0,0.000000,-0.174350
Lebanon,3.840818,1061.041667,4.908693e+06,405.731589,5.833333,0.152143,0.001963,3.840818,1061.041667,4.908693e+06,...,6.0,0.338424,0.002998,-1.702175,742.0,6.859408e+06,535.957905,6.0,0.000023,-0.117882


In [21]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))